# Hometask 7 #

### Задача: 
запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Будем использовать усеченный словарь для ускорения алгоритма, т.к. наша задача найти наиболее часто встречающиеся слова в каждой из тем (тэгов)

In [199]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True, min_df = 25)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=25,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'might', 'always', 'its', 'whole', 'about', 'another', 'cry', 'who', 'rather', 'behind', 'even', 'some', 'name', 'an', 'all', 'few', 'couldnt', 'very', 'anyone', 'on', 'must', 'moreover', 'cant', 'whether', 'before', 'hasnt', 'nobody', 'full', 'keep', 'being', 'with', 'thereaft..., 'own', 'during', 'along', 'will', 'within', 'such', 'between', 'have', 'now', 'several', 'third'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [200]:
len(vectorizer.vocabulary_)

4983

Сэмплирование по данному распределению

In [50]:
def generate_with_weight(weights):
    #return the position with respect to its weight
    weights_normed = np.sort(weights) / np.sum(weights)
    weights_bounded = np.cumsum(weights_normed)

    rand = np.random.rand()
    for i in range(len(weights)):
        if(rand < weights_bounded[i]):
            rand = np.argsort(weights)[i]
            break;
    return rand

In [237]:
### main part
word_to_topic = np.zeros(len(vectorizer.vocabulary_), dtype = int) # z     -- in which topic the word
num_topic = np.zeros(len(newsgroups_train.target_names))           # n_k   -- number of words in topic k
num_topic_word = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))   # <-- n_k,w
                                                                   # n_k,w -- amount of times word w was assigned with topic k
num_text_topic = np.zeros((len(newsgroups_train.data), len(newsgroups_train.target_names)))    # <-- n_d,k
                                                                   # n_d,k -- number of words assigned with topic k in text d

alpha = np.zeros(len(newsgroups_train.target_names))               # alpha ~ distribution of topics over texts 
beta = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))  
                                                                   # beta  ~ distribution of topics over words


# random connect words with topics
for i in range(len(vectorizer.vocabulary_)):       
    # because len(newsgroups_train.target_names) =  20
    word_to_topic[i] = generate_with_weight(np.full(20, 1/20))

# update counters    
for i in range(len(newsgroups_train.data)):
    alpha[newsgroups_train.target[i]] = alpha[newsgroups_train.target[i]] + 1
    text = newsgroups_train.data[i]
    beta[newsgroups_train.target[i]] = beta[newsgroups_train.target[i]] + vectorizer.transform([text])
    
    x = np.resize(vectorizer.transform([text]).toarray(), len(vectorizer.vocabulary_))
    b = np.argwhere(x)
    c = word_to_topic[b]
    for j in range(len(num_topic)):
        num_text_topic[i, j] = len(c[(c == j)])
        num_topic[j] = num_topic[j] + len(c[(c == j)])
    text_transformed = vectorizer.inverse_transform(vectorizer.transform([text]))[0]
    for j in range(len(text_transformed)):
        word = vectorizer.vocabulary_.get(text_transformed[j])
        num_topic_word[word_to_topic[word], word] = num_topic_word[word_to_topic[word], word] + 1
#num_corpus_topic = np.sum(num_text_topic, axis = 0)


## Algorithm
for count in range(150):                                            # will go several times through the corpus for stability
    for i in range(len(newsgroups_train.data)):
        text = newsgroups_train.data[i]
        text_transformed = vectorizer.inverse_transform(vectorizer.transform([text]))[0]  # --[0] because of [text]
        for j in range(len(text_transformed)):
            # change counters
            word = vectorizer.vocabulary_.get(text_transformed[j]) # word index in vocabulary_
            topic = word_to_topic[word]
            num_text_topic[i, topic] = num_text_topic[i, topic] - 1
            num_topic[topic] = num_topic[topic] - 1
            num_topic_word[topic, word] = num_topic_word[topic, word] - 1
            #
            p = np.zeros(len(num_topic))
            for k in range(len(num_topic)):
                # formula:  p[k] = (n_d,k + alpha[k]) * (n[k,w_i] + beta_w_i) / (n_k + beta_sum)
                p[k] = (num_text_topic[i, k] + alpha[k]) * (num_topic_word[k, word] + beta[k, word]) / (num_topic[k] + 
                                                                                                        np.sum(beta[k]))
                #
            # sample new smth
            topic = generate_with_weight(np.abs(p))
            word_to_topic[word] = topic
            # change counters back
            num_text_topic[i, topic] = num_text_topic[i, topic] + 1
            num_topic[topic] = num_topic[topic] + 1
            num_topic_word[topic, word] = num_topic_word[topic, word] + 1
            #

In [238]:
# top 10 words with respect to theirs changable counter from the algorithm
inverse_dict = {v:k  for k,v in vectorizer.vocabulary_.items()}

for i in range(len(newsgroups_train.target_names)):
    print('Top 10 in Topic = {0}'.format(newsgroups_train.target_names[i]))
    print()
    #x = np.argsort(num_topic_word[i])[:-10:-1]
    x = np.argsort(num_topic_word[i]) [word_to_topic[np.argsort(num_topic_word[i])] == i] [:-10:-1]
    for j in range(len(x)):
        print(inverse_dict.get(x[j]), end = ' ')
    print()
    #print(x, sep = '\n')
    print('--------------------------------------------------------------------\n')

Top 10 in Topic = alt.atheism

way doesn read actually yes program remember instead news 
--------------------------------------------------------------------

Top 10 in Topic = comp.graphics

want 24 past wants mark reasonable wondering issues united 
--------------------------------------------------------------------

Top 10 in Topic = comp.os.ms-windows.misc

things kind left nice advance type play argument strong 
--------------------------------------------------------------------

Top 10 in Topic = comp.sys.ibm.pc.hardware

using says stuff version understand area live standard consider 
--------------------------------------------------------------------

Top 10 in Topic = comp.sys.mac.hardware

long thing world second hand john gets aren children 
--------------------------------------------------------------------

Top 10 in Topic = comp.windows.x

good really 10 high software came national single purpose 
--------------------------------------------------------------------



In [239]:
# top 10 words with respect to theirs frequency in specific texts
for i in range(len(newsgroups_train.target_names)):
    print('Top 10 in Topic = {0}\n'.format(newsgroups_train.target_names[i]))
    x = np.argsort(beta[i]) [word_to_topic[np.argsort(beta[i])] == i] [:-10:-1]
    for j in range(len(x)):
        print(inverse_dict.get(x[j]), end = ' ')
    print()
    #print(x, sep = '\n')
    print('--------------------------------------------------------------------\n')

Top 10 in Topic = alt.atheism

way believe atheism read doesn yes actually law accept 
--------------------------------------------------------------------

Top 10 in Topic = comp.graphics

use just looking hi mail want gif send 24 
--------------------------------------------------------------------

Top 10 in Topic = comp.os.ms-windows.misc

file files drivers cica advance directory printer nt things 
--------------------------------------------------------------------

Top 10 in Topic = comp.sys.ibm.pc.hardware

pc using dos scsi motherboard data cards problems bios 
--------------------------------------------------------------------

Top 10 in Topic = comp.sys.mac.hardware

mac quadra disk speed simms price se internal info 
--------------------------------------------------------------------

Top 10 in Topic = comp.windows.x

thanks mit user widget manager example trying software error 
--------------------------------------------------------------------

Top 10 in Topic = misc.f

Как видно из представленных выше наборов слов алгоритм достаточно быстро (алгоритм произвел менее 200 итераций) ставит в соответствие каждому слову определенный тэг, который на самом деле соответствует основной тематике слова. Напротив, ~200 итераций явно недостотаточно для формирования распределения тэгов над словами, удовлетворящего критерию стабильности и нашему представлению о смысле слов. Это происходит в следствие того, что в алгоритме на каждом шаге счётчики меняются на 1, в то время как объём слов помноженный на количество тэгов (даже для усеченного словаря) слишком велик. Таким образом, для получения осмысленных результатов для распределения тэгов над словами необходимо провести намного больше итераций.

=============================================================================================================

#### Extras

Примерное время выполнения шага алгоритма:

In [233]:
%%time
for count in range(10):                                            # will go more times through the corpus for stability
    for i in range(len(newsgroups_train.data)):
        text = newsgroups_train.data[i]
        text_transformed = vectorizer.inverse_transform(vectorizer.transform([text]))[0]  # --[0] because of [text]
        for j in range(len(text_transformed)):
            # change counters
            word = vectorizer.vocabulary_.get(text_transformed[j]) # word index in vocabulary_
            topic = word_to_topic[word]
            num_text_topic[i, topic] = num_text_topic[i, topic] - 1
            num_topic[topic] = num_topic[topic] - 1
            num_topic_word[topic, word] = num_topic_word[topic, word] - 1
            #
            p = np.zeros(len(num_topic))
            for k in range(len(num_topic)):
                # formula:  p[k] = (n_d,k + alpha[k]) * (n[k,w_i] + beta_w_i) / (n_k + beta_sum)
                p[k] = (num_text_topic[i, k] + alpha[k]) * (num_topic_word[k, word] + beta[k, word]) / (num_topic[k] + 
                                                                                                        np.sum(beta[k]))
                #
            # sample new smth
            topic = generate_with_weight(np.abs(p))
            word_to_topic[word] = topic
            # change counters back
            num_text_topic[i, topic] = num_text_topic[i, topic] + 1
            num_topic[topic] = num_topic[topic] + 1
            num_topic_word[topic, word] = num_topic_word[topic, word] + 1
            #


Wall time: 16min 50s


In [201]:
%%time

word_to_topic = np.zeros(len(vectorizer.vocabulary_), dtype = int) # z     -- in which topic the word
num_topic = np.zeros(len(newsgroups_train.target_names))           # n_k   -- number of words in topic k
num_topic_word = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))   # <-- n_k,w
                                                                   # n_k,w -- amount of times word w was assigned with topic k
num_text_topic = np.zeros((len(newsgroups_train.data), len(newsgroups_train.target_names)))    # <-- n_d,k
                                                                   # n_d,k -- number of words assigned with topic k in text d
#num_corpus_topic                                                  # in algorithm NOT use n_d,k as d is all texts together

alpha = np.zeros(len(newsgroups_train.target_names))               # alpha ~ distribution of topics over texts 
beta = np.zeros((len(newsgroups_train.target_names), len(vectorizer.vocabulary_)))  
                                                                   # beta  ~ distribution of topics over words


# random connect words with topics
for i in range(len(vectorizer.vocabulary_)):       
    # because len(newsgroups_train.target_names) =  20
    word_to_topic[i] = generate_with_weight(np.full(20, 1/20))

# update counters    
for i in range(len(newsgroups_train.data)):
    alpha[newsgroups_train.target[i]] = alpha[newsgroups_train.target[i]] + 1
    text = newsgroups_train.data[i]
    beta[newsgroups_train.target[i]] = beta[newsgroups_train.target[i]] + vectorizer.transform([text])
    
    x = np.resize(vectorizer.transform([text]).toarray(), len(vectorizer.vocabulary_))
    b = np.argwhere(x)
    c = word_to_topic[b]
    for j in range(len(num_topic)):
        num_text_topic[i, j] = len(c[(c == j)])
        num_topic[j] = num_topic[j] + len(c[(c == j)])
    text_transformed = vectorizer.inverse_transform(vectorizer.transform([text]))[0]
    for j in range(len(text_transformed)):
        word = vectorizer.vocabulary_.get(text_transformed[j])
        num_topic_word[word_to_topic[word], word] = num_topic_word[word_to_topic[word], word] + 1

Wall time: 20.7 s
